<a href="https://colab.research.google.com/github/themysterysolver/QR-CIP/blob/main/CODE/PHASE_1_REFINED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy matplotlib opencv-python qrcode[pil] pycryptodome pillow

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import qrcode
import hashlib

- `cv2` is a part of *openCV* ***computer vision library***

In [7]:
def sha256_to_float(seed_string):
    hash_digest = hashlib.sha256(seed_string.encode()).hexdigest()
    print(hash_digest)
    hash_n = hash_digest[:16]
    return int(hash_n, 16) / 2**64

In [8]:
def lss_permutation(seed_string, n, r=3.9, s=3.0):#s is amplitutde,r-is logistic growth rate,pi-freq paramaeter
    x = sha256_to_float(seed_string)
    seq=[0]*n
    seq[0]=x
    for i in range(1,n):
      seq[i]=((r*seq[i-1]*(1-seq[i-1])) + s*np.sin(np.pi*seq[i-1]))%1
    permu_seq=np.argsort(seq)
    return np.array(permu_seq)

In [11]:
def crop_qr_border(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return img  # If no contours found, return the original image
    cnt = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(cnt)
    margin = 5
    x = max(0, x - margin)
    y = max(0, y - margin)
    w = min(img.shape[1] - x, w + 2 * margin)
    h = min(img.shape[0] - y, h + 2 * margin)
    return img[y:y+h, x:x+w]

In [15]:
def divide_qr(img):
    if img.shape[0] % 4 != 0 or img.shape[1] % 4 != 0:
        img = cv2.resize(img, (400, 400))
    block_size = img.shape[0] // 4
    return np.array([img[i * block_size:(i + 1) * block_size, j * block_size:(j + 1) * block_size] for i in range(4) for j in range(4)])

In [16]:
def scramble_qr(blocks, permutation):
    return blocks[permutation.tolist()] #converts into list

In [17]:
def rebuild_matrix(blocks):
    return np.vstack([np.hstack(blocks[i * 4:(i + 1) * 4]) for i in range(4)])

In [19]:
if __name__ == "__main__":
    sha256_input = input("Enter a string for SHA-256-based permutation:")
    n=16
    permutation=lss_permutation(sha256_input,n)
    print(permutation)

    qr_input = input("Enter a string to generate a QR code: ")
    qr=qrcode.make(qr_input)
    qr.save("qr_code.png")

    img=cv2.imread("qr_code.png")
    print(img.shape)

    img = crop_qr_border(img)
    print(img.shape)

    img = cv2.resize(img, (400, 400))
    print(img.shape)

    divided_blocks = divide_qr(img)
    print(divided_blocks.shape)

    scrambled_blocks = scramble_qr(divided_blocks, permutation)
    print(scrambled_blocks.shape)

    scrambled_qr = rebuild_matrix(scrambled_blocks)
    print(scrambled_qr.shape)



Enter a string for SHA-256-based permutation:hello
2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824
[ 6  1  0 13  3 14  7  4 10 15  8 11  2 12  5  9]
Enter a string to generate a QR code: wow
(290, 290, 3)
(220, 210, 3)
(400, 400, 3)
(16, 100, 100, 3)
(16, 100, 100, 3)
(400, 400, 3)


- That's how a `290*290*3` array look like,290 rows and column with 3 cols each for ***rgb***
```
[
  [[255, 255, 255], [255, 255, 255], [255, 255, 255], ..., [255, 255, 255]],
  [[255, 255, 255], [0, 0, 0], [0, 0, 0], ..., [255, 255, 255]],
  [[255, 255, 255], [0, 0, 0], [0, 0, 0], ..., [255, 255, 255]],
  ...,
  [[255, 255, 255], [0, 0, 0], [0, 0, 0], ..., [255, 255, 255]]
]
```